In [1]:
import pandas as pd
import numpy
import requests
import bs4
import time
from tqdm import tqdm
import re

In [2]:
# Getting single page
url = 'https://fbref.com/en/squads/cff3d9bb/2018-2019/matchlogs/all_comps/shooting/Chelsea-Match-Logs-All-Competitions'

response = requests.get(url)
response.encoding = 'utf-8'
soup = bs4.BeautifulSoup(response.text, "lxml")

In [3]:
# Getting the table
tble = soup.find_all('table')[0]

In [4]:
# Getting table headers
headers = []
for i in tble.find_all('thead')[0].find_all('tr')[1].children:
    try:
        headers.append(str(i.text))
    except:
        continue
# headers = [str(i) for i in tble.find_all('thead')[0].find_all('tr')[1].children]
headers[:3]

['Date', 'Time', 'Comp']

In [5]:
all_rows = []
content = tble.find_all('tbody')[0]
for row in content.find_all('tr', attrs = {'class': 'side-for'}):
    a_row = []
    for c in row.children:
        a_row.append(c.text)
    all_rows.append(a_row)

In [6]:
pd.DataFrame(all_rows, columns = headers)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2018-08-05,15:00,Community Shield,FA Community Shield,Sun,Neutral,L,0,2,Manchester City,...,,,0,0,,,,,,Match Report
1,2018-08-11,15:00,Premier League,Matchweek 1,Sat,Away,W,3,0,Huddersfield,...,15.7,2,1,1,1.9,1.1,0.10,+1.1,+0.9,Match Report
2,2018-08-18,17:30,Premier League,Matchweek 2,Sat,Home,W,3,2,Arsenal,...,20.1,1,0,0,2.3,2.3,0.10,+0.7,+0.7,Match Report
3,2018-08-26,16:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Newcastle Utd,...,17.7,0,1,1,1.5,0.8,0.05,-0.5,-0.8,Match Report
4,2018-09-01,15:00,Premier League,Matchweek 4,Sat,Home,W,2,0,Bournemouth,...,19.2,1,0,0,1.4,1.4,0.06,+0.6,+0.6,Match Report
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,2019-05-02,21:00,Europa Lg,Semi-finals,Thu,Away,D,1,1,de Eint Frankfurt,...,18.7,1,0,0,1.0,1.0,0.06,0.0,0.0,Match Report
59,2019-05-05,14:00,Premier League,Matchweek 37,Sun,Home,W,3,0,Watford,...,14.3,0,0,0,1.9,1.9,0.11,+1.1,+1.1,Match Report
60,2019-05-09,20:00,Europa Lg,Semi-finals,Thu,Home,W,1 (4),1 (3),de Eint Frankfurt,...,19.1,0,4,5,1.9,0.9,0.07,+3.1,+0.1,Match Report
61,2019-05-12,15:00,Premier League,Matchweek 38,Sun,Away,D,0,0,Leicester City,...,19.1,0,0,0,1.5,1.5,0.11,-1.5,-1.5,Match Report


In [7]:
def parse_single_page(url):
#     print('Parsing:', url)
    # Get http response
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = bs4.BeautifulSoup(response.text, "lxml")
    
    # Get the table
    tble = soup.find_all('table')[0]
    
    # Get table header
    headers = []
    for i in tble.find_all('thead')[0].find_all('tr')[1].children:
        try:
            headers.append(str(i.text))
        except:
            continue
            
    # Get table content
    all_rows = []
    content = tble.find_all('tbody')[0]
    for row in content.find_all('tr', attrs = {'class': 'side-for'}):
        a_row = []
        for c in row.children:
            a_row.append(c.text)
        all_rows.append(a_row)

    return pd.DataFrame(all_rows, columns = headers)

In [8]:
parse_single_page('https://fbref.com/en/squads/206d90db/2018-2019/matchlogs/all_comps/schedule/Barcelona-Scores-and-Fixtures-All-Competitions')



IndexError: list index out of range

In [ ]:
# url_head = 'https://fbref.com/en/squads/cff3d9bb/2018-2019/matchlogs/all_comps/'
# url_tail = '/Chelsea-Match-Logs-All-Competitions'

In [73]:
keys = ['shooting', 'passing_types', 'gca', 'possession', 'misc']
years = range(2014, 2020)
data_dir = 'data/'

url_base_ba = 'https://fbref.com/en/squads/206d90db/%s/matchlogs/all_comps/%s/Barcelona-Match-Logs-All-Competitions'
url_base_ch = 'https://fbref.com/en/squads/cff3d9bb/%s/matchlogs/all_comps/%s/Chelsea-Match-Logs-All-Competitions'

for y in tqdm(years):
    for k in keys:
        year_str = str(y) + '-' + str(y+1)
        
        # Barcelona
        url = url_base_ba % (year_str, k)
        tmp_df = parse_single_page(url)
        tmp_df.to_csv(data_dir + 'barca_' + str(y) + '_' + k + '.csv', index = False)
        
        # Chelsea
        url = url_base_ch % (year_str, k)
        tmp_df = parse_single_page(url)
        tmp_df.to_csv(data_dir + 'chelsea_' + str(y) + '_' + k + '.csv', index = False)

100%|██████████| 6/6 [00:35<00:00,  5.85s/it]


In [74]:
url = 'http://ft.10jqka.com.cn/thsft/attachment.php?pubType=009003002007001&snapshot=1&id=D9363362.htm'

In [84]:
response = requests.get(url)
# response.encoding = 'utf-8'
soup = bs4.BeautifulSoup(response.text, "lxml")

In [85]:
soup.find('item')

In [86]:
for i in soup(text=re.compile(r'Item ')):
    print(i)

Item 2.03. Creation of a Direct Financial Obligation or an Obligation under an Off-Balance Sheet Arrangement of a Registrant
Item 9.01.  Financial Statements and Exhibits


In [87]:
soup.find_all(text = re.compile('Item '))

['Item 2.03. Creation of a Direct Financial Obligation or an Obligation under an Off-Balance Sheet Arrangement of a Registrant',
 'Item 9.01.  Financial Statements and Exhibits']